# Projeto 1 - Ciência dos Dados

Nome: Raphael Lahiry de Barros

Nome: Florencia Averame Kramer

___
## Carregando algumas bibliotecas:

In [111]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from math import *

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [112]:
import os

filename = 'ps4.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo ps4.xlsx, tudo certo para prosseguir com a prova!


### Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [113]:
train = pd.read_excel(filename)
train.head()

,Treinamento,Relevância
0,@odonodocabare_ comprei um tablet dia desses e...,0.0
1,@encanadorchapad o negócio é o ps4 slim. peque...,4.0
2,rt @ps3brasil: tohu é anunciado para ps4; deta...,2.0
3,dividindo essa semana entre dois jogos no modo...,0.0
4,@leobruto @fridaygus @douglasxavier8 @ocerveje...,0.0


In [114]:
train['Treinamento'] = train['Treinamento'].astype('str')

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Nosso produto é o playstation 4. Consideramos relevantes as mensagens que falavam sobre questões técnicas do produto e sobre satisfação, sejam positivas ou negativas. Consideramos irrelevantes as mensagens que só mencionavam o nosso produtos nas '#' ou que eram opiniões pessoais que não agregavam na nossa análise.

___
## Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [115]:
#função que apaga os caracteres especiais e as menções de usuários
# https://docs.python.org/3/library/re.html#
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;"\n"()''"",_%$\|/,<>]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    
    #filtro que remove as menções à usuários com @
    filtro = filter(lambda x:x[0]!='@', text_subbed.split())
    juncao = " ".join(filter(lambda x:x[0]!='@', text_subbed.split()))
    
    return juncao
    

In [116]:
#aplicando a função cleanup na base treinamento e deixando tudo em letra minúscula
train['Treinamento'] = train['Treinamento'].apply(cleanup)
train['Treinamento'] = train['Treinamento'].str.lower()

In [117]:
#classificando a coluna Relevância como uma variável qualitativa ordinal
train['Relevância'] = train['Relevância'].astype('category')
train['Relevância'].cat.categories = ['Muito irrelevante', 'Irrelevante', 'Neutro', 'Relevante', 'Muito relevante']
#train['Relevância'].cat.as_ordered()

### Criando os filtros para criar os dataFrames por relevância

In [118]:
muito_relevante = train['Relevância'] == 'Muito relevante'
relevante = train['Relevância'] == 'Relevante'
neutro = train['Relevância'] == 'Neutro'
irrelevante = train['Relevância'] == 'Irrelevante'
muito_irrelevante = train['Relevância'] == 'Muito irrelevante'

### Criando os dataFrames separados por relevância

In [119]:
df_muito_relevante = train.loc[muito_relevante, :]
df_relevante = train.loc[relevante, :]
df_neutro = train.loc[neutro, :]
df_irrelevante = train.loc[irrelevante, :]
df_muito_irrelevante = train.loc[muito_irrelevante, :]

### Separando cada dataFrame da sua respectiva relevância em listas de palavras e em listas de tweets

In [120]:
tweets_muito_relevantes = []
lista_vrw = []
for vrw in df_muito_relevante['Treinamento']:
    tweets_muito_relevantes.append(vrw.split(' '))
for vrl in tweets_muito_relevantes:
    lista_vrw += vrl

In [121]:
tweets_relevantes = []
lista_rw = []
for rw in df_relevante['Treinamento']:
    tweets_relevantes.append(rw.split(' '))
for rl in tweets_relevantes:
    lista_rw += rl

In [122]:
tweets_neutros = []
lista_nw = []
for nw in df_neutro['Treinamento']:
    tweets_neutros.append(nw.split(' '))
for nl in tweets_neutros:
    lista_nw += nl

In [123]:
tweets_irrelevantes = []
lista_iw = []
for iw in df_irrelevante['Treinamento']:
    tweets_irrelevantes.append(iw.split(' '))
for il in tweets_irrelevantes:
    lista_iw += il

In [124]:
tweets_muito_irrelevantes = []
lista_viw = []
for viw in df_muito_irrelevante['Treinamento']:
    tweets_muito_irrelevantes.append(viw.split(' '))
for vil in tweets_muito_irrelevantes:
    lista_viw += vil

### Transformando as listas de palavras de cada relevância em pd.Series

In [125]:
serie_muito_relevante = pd.Series(lista_vrw)
serie_relevante = pd.Series(lista_rw)
serie_neutro = pd.Series(lista_nw)
serie_irrelevante = pd.Series(lista_iw)
serie_muito_irrelevante = pd.Series(lista_viw)

### Definindo os parâmetros para realizar o 'smoothing'

In [126]:
#alpha é um "chute" da frequência absoluta das palavras que não estão em nenhum tweet classificado
#utilizamos a variante Laplace Smoothing (alpha = 1)
alpha = 1

#v é um "chute" da quantidade de palavras existentes na língua potuguesa que não estão nos tweets classificados
v = 1e4

### Frequências relativas e absolutas das palavras de cada relevância

In [127]:
#muito relevante
freq_ab_mr = serie_muito_relevante.value_counts() 
freq_re_mr = (freq_ab_mr + alpha)/(freq_ab_mr.sum() + (alpha*v))

In [128]:
#relevante
freq_ab_r = serie_relevante.value_counts() 
freq_re_r = (freq_ab_r + alpha)/(freq_ab_r.sum() + (alpha*v))

In [129]:
#neutro
freq_ab_n = serie_neutro.value_counts() 
freq_re_n = (freq_ab_n + alpha)/(freq_ab_n.sum() + (alpha*v))

In [130]:
#irrelevante
freq_ab_i = serie_irrelevante.value_counts() 
freq_re_i = (freq_ab_i + alpha)/(freq_ab_i.sum() + (alpha*v)) 

In [131]:
#muito irrelevante
freq_ab_mi = serie_muito_irrelevante.value_counts() 
freq_re_mi = (freq_ab_mi + alpha)/(freq_ab_mi.sum() + (alpha*v))

## Montando Naive-Bayes

### Probabilidade de cada relevância à priori

In [132]:
#contagem de tweets
p_mr = len(tweets_muito_relevantes) / len(train["Treinamento"])
p_r = len(tweets_relevantes) / len(train["Treinamento"])
p_n = len(tweets_neutros) / len(train["Treinamento"])
p_i = len(tweets_irrelevantes) / len(train["Treinamento"])
p_mi = len(tweets_muito_irrelevantes) / len(train["Treinamento"])

print(p_mr + p_r + p_n + p_i + p_mi)

0.9986684420772303


### Função que retorna a probabilidade de uma frase pertencer à uma cateogria específica de relevância com base nas palavras que a compõem

In [133]:
def datawitter(frase):
    
    prob_frase_dado_mr = 1
    prob_frase_dado_r = 1
    prob_frase_dado_n = 1
    prob_frase_dado_i = 1
    prob_frase_dado_mi = 1
    
    frase = frase.split(' ')
    
    for palavra in frase:
        if palavra in freq_re_mr:
            prob_frase_dado_mr *= freq_re_mr[palavra]
        elif palavra not in freq_re_mr:
            prob_frase_dado_mr *= (alpha/(freq_ab_mr.sum() + (alpha*v))) 
            
        if palavra in freq_re_r:
            prob_frase_dado_r *= freq_re_r[palavra]
        elif palavra not in freq_re_r:
            prob_frase_dado_r *= (alpha/(freq_ab_r.sum() + (alpha*v)))      
            
        if palavra in freq_re_n:
            prob_frase_dado_n *= freq_re_n[palavra]
        elif palavra not in freq_re_n:
            prob_frase_dado_n *= (alpha/(freq_ab_n.sum() + (alpha*v)))
            
        if palavra in freq_re_i:
            prob_frase_dado_i *= freq_re_i[palavra]
        elif palavra not in freq_re_i:
            prob_frase_dado_i *= (alpha/(freq_ab_i.sum() + (alpha*v)))
            
        if palavra in freq_re_mi:
            prob_frase_dado_mi *= freq_re_mi[palavra]
        elif palavra not in freq_re_mi:
            prob_frase_dado_mi *= (alpha/(freq_ab_mi.sum() + (alpha*v)))
            
       
    prob_mr_dado_frase = prob_frase_dado_mr*p_mr
 
    prob_r_dado_frase = prob_frase_dado_r*p_r

    prob_n_dado_frase = prob_frase_dado_n*p_n

    prob_i_dado_frase = prob_frase_dado_i*p_i

    prob_mi_dado_frase = prob_frase_dado_mi*p_mi
    
    
    if max(log(prob_mr_dado_frase, 10), log(prob_r_dado_frase, 10), log(prob_n_dado_frase, 10), log(prob_i_dado_frase, 10),  log(prob_mi_dado_frase, 10)) == log(prob_mr_dado_frase, 10):
        return 'Muito relevante'
    if max(log(prob_mr_dado_frase, 10), log(prob_r_dado_frase, 10), log(prob_n_dado_frase, 10), log(prob_i_dado_frase, 10),  log(prob_mi_dado_frase, 10)) == log(prob_r_dado_frase, 10): 
        return 'Relevante'
    if max(log(prob_mr_dado_frase, 10), log(prob_r_dado_frase, 10), log(prob_n_dado_frase, 10), log(prob_i_dado_frase, 10),  log(prob_mi_dado_frase, 10)) == log(prob_n_dado_frase, 10): 
        return 'Neutro'
    if max(log(prob_mr_dado_frase, 10), log(prob_r_dado_frase, 10), log(prob_n_dado_frase, 10), log(prob_i_dado_frase, 10),  log(prob_mi_dado_frase, 10)) == log(prob_i_dado_frase, 10):
        return 'Irrelevante'
    if max(log(prob_mr_dado_frase, 10), log(prob_r_dado_frase, 10), log(prob_n_dado_frase, 10), log(prob_i_dado_frase, 10),  log(prob_mi_dado_frase, 10)) == log(prob_mi_dado_frase, 10): 
        return 'Muito irrelevante'

### Preenchimento da coluna contém a nossa classificação

In [134]:
train['Classificacao'] = train['Treinamento'].apply(datawitter)

### Tabela que compara as classificações que nós fizemos previamente à mão com as classificações dadas pelo nosso classificador

In [135]:
tabela = pd.crosstab(train['Relevância'], train['Classificacao'], normalize='all')
tabela

Classificacao,Irrelevante,Muito irrelevante,Muito relevante,Neutro,Relevante
Relevância,,,,,
Muito irrelevante,0.053333,0.214667,0.000000,0.009333,0.001333
Irrelevante,0.290667,0.004000,0.000000,0.006667,0.000000
Neutro,0.017333,0.001333,0.000000,0.192000,0.000000
Relevante,0.017333,0.001333,0.000000,0.004000,0.137333
Muito relevante,0.029333,0.000000,0.005333,0.009333,0.005333


### Eficácia do nosso classificador

In [136]:
#quanto o classificador acertou?
f'{(tabela.iloc[0,1] + tabela.iloc[1,0] + tabela.iloc[2,3] + tabela.iloc[3,4] + tabela.iloc[4,2])*100:.5g}%'

'84%'

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

### Transformando o dataFrame teste em string para poder aplicar a função cleanup e deixar o texto todo em minúscula

In [137]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test['Teste'] = test['Teste'].astype('str')
test['Teste'] = test['Teste'].apply(cleanup)
test['Teste'] = test['Teste'].str.lower()

### Transformando a coluna 'Relevância' em categórica

In [138]:
test['Relevância'] = test['Relevância'].astype('category')
test['Relevância'].cat.categories = ['Muito irrelevante', 'Irrelevante', 'Neutro', 'Relevante', 'Muito relevante']

### Preenchendo a coluna que contêm as classificações do nosso classificador

In [ ]:
test['Classificador'] = test['Teste'].apply(datawitter)

### Tabela que compara as classificações que nós fizemos previamente à mão com as classificações dadas pelo nosso classificador

In [ ]:
tabela2 = pd.crosstab(test['Relevância'], test['Classificador'], normalize='all')
tabela2

### Eficácia do nosso classificador

In [ ]:
#quanto o classificador acertou?
#o último 0 representa que não há tweets classificados como muito relevantes no nosso teste
f'O acerto do nosso classificador foi de: {(tabela2.iloc[0,1]+tabela2.iloc[1,0]+tabela2.iloc[2,2]+tabela2.iloc[3,3]+0)*100:.5g}%'

___
# Concluindo

#### Porcentagem de verdadeiros muito relevantes e falsos muito relevantes

In [ ]:
print(f'Porcentagem de mensagens muito relevantes que foram classificadas como muito relevantes: {(0)}%')
print()
print(f'Porcentagem de mensagens muito relevantes que não foram classificadas como muito relevantes: {(1)*100:.5g}%')

#### Porcentagem de verdadeiros relevantes e falsos relevantes

In [ ]:
print(f'Porcentagem de mensagens relevantes que foram classificadas como relevantes: {(tabela2.iloc[3,3])*100:.5g}%')
print()
print(f'Porcentagem de mensagens relevantes que não foram classificadas como relevantes: {(1-tabela2.iloc[3,3])*100:.5g}%')

#### Porcentagem de verdadeiros neutros e falsos neutros

In [ ]:
print(f'Porcentagem de mensagens neutras que foram classificadas como neutras: {(tabela2.iloc[2,2])*100:.5g}%')
print()
print(f'Porcentagem de mensagens neutras que não foram classificadas como neutras: {(1-tabela2.iloc[2,2])*100:.5g}%')

#### Porcentagem de verdadeiros irrelevantes e falsos irrelevantes

In [ ]:
print(f'Porcentagem de mensagens irrelevantes que foram classificadas como irrelevantes: {(tabela2.iloc[1,0])*100:.5g}%')
print()
print(f'Porcentagem de mensagens irrelevantes que não foram classificadas como irrelevantes: {(1-tabela2.iloc[1,0])*100:.5g}%')

#### Porcentagem de verdadeiros muito irrelevantes e falsos muito irrelevantes

In [ ]:
print(f'Porcentagem de mensagens muito irrelevantes que foram classificadas como muito irrelevantess: {(tabela2.iloc[0,1])*100:.5g}%')
print()
print(f'Porcentagem de mensagens muito irrelevantes que não foram classificadas como muito irrelevantes: {(1-tabela2.iloc[0,1])*100:.5g}%')

### Como são tratadas as mensagens com dupla negação e sarcasmo?
Como o classificador Naïve-Bayes trabalha exclusivamente com as probabilidades das palavras de uma frase pertencerem a uma certa categoria, ele não leva em consideração a ordem em que essas palavras acontecem, desta forma, não conseguindo analisar se uma frase possui sarcasmo ou dupla negação. Esse é um dos motivos pelos quais o classificador recebe o nome 'ingênuo (naive)'. 

### Por que não posso usar o próprio classificador para gerar mais amostras de treinamento?
Não podemos utilizar o próprio classificador para gerar mais amostras de treinamento porque o classificador foi feito apartir das classificações de treinamento. Isso faz com que a eficácia do classificador se mostre como sendo muito alta, podendo não ser a realidade, já que os tweets que serão classificados são os que geraram os 'padrões' que criaram o classifcador.

### Diferentes cenários para Naïve Bayes fora do contexto do projeto 
O classificador Naïve-Bayes também poderia ser utilizado em uma campanha política, onde se tem por objetivo saber o apoio que certo candidato está recebendo através das classificações das mensagens que o mencionam nas redes sociais (não só twitter).
De um modo geral, o classificador Naïve-Bayes pode ser muito útil para se realizar uma análise automática de sentimento relacionada a qualquer produto/ marca/ pessoa. É um modo de se obter feedbacks de forma informal e sincera.

### Melhorias possíveis para o classificador Naïve-Bayes
Poderiamos obter uma classificação mais realista ao levar em consideração a ordem na qual as palavras aparecem nas frases e a probabilidade de certas sequências de palavras acontecerem. Com isso, poderiamos analisar a categoria de relevância / positividade / negativade da mensagem através da probailidade de que certa sequência se enquadre em algum desses cenários. 

1) https://www.organicadigital.com/blog/algoritmo-de-classificacao-naive-bayes/
2) https://sebastianraschka.com/Articles/2014_naive_bayes_1.html

### Dashboard que realiza análise de sentimento e visualiza estes dados

In [ ]:
dado = test['Classificador'].value_counts(True)

colors = ['orange', 'yellow', 'green', 'red']

dado.plot.bar(stacked = True, color=colors)
    
plt.xlabel('\nRelevância dos tweets "Teste"\n')
plt.ylabel('\nPorcentagem dos tweets "Teste"\n')
plt.title('\nAnálise de sentimento do classificador\n')
plt.show()

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**